# Data visualization

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import sys
sys.path.append('../fpgp')
import plotting_tools
import utils
import matplotlib.pyplot as plt
import os.path

from deap import tools

import numpy as np
import pickle

## Load data

In [ ]:
exp_dir = '../data/f2'
logs, names = utils.get_results(exp_dir)

In [ ]:
print(names)

In [ ]:
logs = [logs[2], logs[1]]
names = [names[2], names[1]]

In [ ]:
data = np.load(os.path.join(exp_dir, names[0], 'dataset.npz'))
trn_x, trn_y = data['trn'][:, :-1], data['trn'][:, -1]

In [ ]:
for log in logs:
    for l in log:
        print(l['path'], l['logbook'].select('test_set_f')[-1], l['logbook'].select('pred_size')[-1])
        plotting_tools.two_stats_graph(l['logbook'], 'test_set_f', 'pred_size')
        plt.show()
        break
    break

In [ ]:
plotting_tools.compare_performance(logs, 'evals', 'pred_size', method_names=names, ignore_tresh=np.inf, xlabel='Evaluations', ylabel='Test set fitness')
plotting_tools.compare_performance(logs, 'evals', 'test_set_f', method_names=names, ignore_tresh=100, xlabel='Evaluations', ylabel='Test set fitness')
plt.show()

In [ ]:
# select runs
runs = list(list(logs)[0])

In [ ]:
runs[0][-1]['best_sol_func']

In [ ]:
plt.hist(trn_y)

## Histogram of test set fitnesses

In [ ]:
data['trn']

In [ ]:
sorted_logs = sorted(runs, key=lambda x: x[-1]['test_set_f'])
# select run you want to analyze
log = sorted_logs[0] # best run

## Performance of the run

In [ ]:
for l in sorted_logs:
    plotting_tools.show_performance(l, 'evals', 'test_set_f')
    print(l.select('test_set_f')[-1])

## Found solution

In [ ]:
fig, ax = plt.subplots()
ax.plot(trn_x, log[-1]['best_sol_vals'], label='best solution found')
ax.plot(trn_x, trn_y, ls=' ', marker='o', ms='1', label='target function')
ax.legend()
plt.show()

## Histogram of used predictors

In [ ]:
plotting_tools.predictor_histogram(trn_x, trn_y, log)
plt.show()

## Animation of the progress of the run

In [ ]:
anim = plotting_tools.visualize_run(trn_x, trn_y, log, freq=50, step=10)
plt.show()